In [1]:
#!/usr/bin/env python
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import astropy.units as u


# do this before importing webb
os.environ["CRDS_PATH"] = "/orange/adamginsburg/jwst/brick/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from jwst.pipeline import calwebb_image3
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from align_to_catalogs import realign_to_vvv, merge_a_plus_b, retrieve_vvv
#from saturated_star_finding import iteratively_remove_saturated_stars, remove_saturated_stars

from destreak import destreak

import crds

import datetime
import jwst

def print(*args, **kwargs):
    now = datetime.datetime.now().isoformat()
    from builtins import print as printfunc
    return printfunc(f"{now}:", *args, **kwargs)


print(jwst.__version__)

# see 'destreak410.ipynb' for tests of this
medfilt_size = {'F410M': 15, 'F405N': 256, 'F466N': 55}

#basepath = '/orange/adamginsburg/jwst/brick/'

filtername = 'F405N'
module = 'merged'
field = '002'
regionname = 'cloudc'

2023-09-06T13:07:41.344128: 1.11.3


In [2]:
calwebb_image3.Image3Pipeline.call??

Signature: calwebb_image3.Image3Pipeline.call(*args, **kwargs)
Source:   
    @classmethod
    def call(cls, *args, **kwargs):
        """
        Creates and runs a new instance of the class.

        Gets a config file from CRDS if one is available

        To set configuration parameters, pass a `config_file` path or
        keyword arguments.  Keyword arguments override those in the
        specified `config_file`.

        Any positional `*args` will be passed along to the step's
        `process` method.

        Note: this method creates a new instance of `Step` with the given
        `config_file` if supplied, plus any extra `*args` and `**kwargs`.
        If you create an instance of a Step, set parameters, and then use
        this `call()` method, it will ignore previously-set parameters, as
        it creates a new instance of the class with only the `config_file`,
        `*args` and `**kwargs` passed to the `call()` method.

        If not used with a `config_file` or spe

In [4]:
vvvcat = fits.open('/orange/adamginsburg/jwst/cloudc//F466N/pipeline/jw02221-o001_t001_nircam_clear-F466N-merged_vvvcat.fits')

In [9]:
vvvcat[1].data

FITS_rec([(266.602823, -28.60222 , 'VVV J174624.67-283607.99', -1, 18.806, 0.084,  0, 17.736, 0.053,  0, 16.528, 0.034, 16,    nan,   nan,  0,    nan,   nan,  0, 266.602823, -28.60222 ),
          (266.606742, -28.598289, 'VVV J174625.61-283553.84', -1, 16.278, 0.011,  0, 15.811, 0.011,  0, 15.225, 0.011, 16, 14.721, 0.027, 16, 14.421, 0.058, 16, 266.606742, -28.598289),
          (266.613329, -28.597491, 'VVV J174627.19-283550.96', -1, 17.865, 0.037, 16, 17.298, 0.036, 16, 16.693, 0.039, 16, 16.261, 0.111, 16,    nan,   nan,  0, 266.613329, -28.597491),
          ...,
          (266.59499 , -28.520139, 'VVV J174622.79-283112.49',  1,    nan,   nan,  0,    nan,   nan,  0, 17.2  , 0.061, 16, 14.584, 0.024, 16,    nan,   nan,  0, 266.59499 , -28.520139),
          (266.592745, -28.520205, 'VVV J174622.25-283112.73', -1,    nan,   nan,  0,    nan,   nan,  0,    nan,   nan,  0,    nan,   nan,  0, 14.23 , 0.049, 16, 266.592745, -28.520205),
          (266.593806, -28.520737, 'VVV J174622.51

In [2]:
from astropy.table import Table

In [16]:

vvvdr2fn = '/orange/adamginsburg/jwst/cloudc//F466N/pipeline/jw02221-o002_t001_nircam_clear-F466N-merged_vvvcat.ecsv'
#(f'{basepath}/{filtername.upper()}/pipeline/jw02221-o{field}_t001_nircam_clear-{filtername}-{module}_vvvcat.ecsv')

os.path.exists(vvvdr2fn)

False

In [13]:
Table(vvvcat[1].data)

RAJ2000,DEJ2000,iauname,mClass,Zmag3,e_Zmag3,Zperrbits,Ymag3,e_Ymag3,Yperrbits,Jmag3,e_Jmag3,Jperrbits,Hmag3,e_Hmag3,Hperrbits,Ksmag3,e_Ksmag3,Ksperrbits,RA,DEC
float64,float64,str24,int16,float32,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64
266.602823,-28.60222,VVV J174624.67-283607.99,-1,18.806,0.084,0,17.736,0.053,0,16.528,0.034,16,nan,nan,0,nan,nan,0,266.602823,-28.60222
266.606742,-28.598289,VVV J174625.61-283553.84,-1,16.278,0.011,0,15.811,0.011,0,15.225,0.011,16,14.721,0.027,16,14.421,0.058,16,266.606742,-28.598289
266.613329,-28.597491,VVV J174627.19-283550.96,-1,17.865,0.037,16,17.298,0.036,16,16.693,0.039,16,16.261,0.111,16,nan,nan,0,266.613329,-28.597491
266.611708,-28.596065,VVV J174626.81-283545.83,1,19.567,0.167,0,19.022,0.17,0,17.988,0.126,0,15.987,0.086,16,14.496,0.062,16,266.611708,-28.596065
266.603496,-28.60398,VVV J174624.83-283614.32,-1,16.636,0.014,0,16.304,0.016,0,15.796,0.018,0,14.848,0.03,16,13.669,0.029,16,266.603496,-28.60398
266.610196,-28.601597,VVV J174626.44-283605.75,-1,18.836,0.087,0,18.411,0.097,0,17.722,0.099,0,17.499,0.345,0,nan,nan,0,266.610196,-28.601597
266.613424,-28.595982,VVV J174627.22-283545.53,1,19.395,0.143,0,18.878,0.149,0,17.766,0.103,16,nan,nan,0,nan,nan,0,266.613424,-28.595982
266.603983,-28.60527,VVV J174624.95-283618.97,1,nan,nan,0,nan,nan,0,15.802,0.018,0,13.3,0.008,16,11.588,0.004,16,266.603983,-28.60527
266.614028,-28.605808,VVV J174627.36-283620.90,-1,18.678,0.075,0,17.789,0.056,0,16.494,0.033,16,14.852,0.03,16,nan,nan,0,266.614028,-28.605808


In [3]:
#def main(filtername, module, Observations=None, regionname='brick', field='001'):
log.info(f"Processing filter {filtername} module {module}")
# sanity check
if regionname == 'brick':
    assert field == '001'
elif regionname == 'cloudc':
    assert field == '002'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
os.environ["CRDS_PATH"] = f"{basepath}/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
# Files created in this notebook will be saved
# in a subdirectory of the base directory called `Stage3`
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'
print('Output directory set as:', output_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
os.chdir(output_dir)
# the files are one directory up
for fn in glob("../*cal.fits"):
    try:
        os.link(fn, './'+os.path.basename(fn))
    except Exception as ex:
        print(f'Failed to link {fn} to {os.path.basename(fn)} because of {ex}')


2023-09-06 13:07:41,751 - stpipe - INFO - Processing filter F405N module merged


INFO: Processing filter F405N module merged [unknown]
2023-09-06T13:07:41.754413: Output directory set as: /orange/adamginsburg/jwst/cloudc/F405N/pipeline/


In [4]:
Observations.cache_location = output_dir
obs_table = Observations.query_criteria(
                                        proposal_id="2221",
                                        proposal_pi="Ginsburg*",
                                        #calib_level=3,
                                        )
print("Obs table length:", len(obs_table))
msk = ((np.char.find(obs_table['filters'], filtername.upper()) >= 0) |
       (np.char.find(obs_table['obs_id'], filtername.lower()) >= 0))
data_products_by_obs = Observations.get_product_list(obs_table[msk])
print("data prodcts by obs length: ", len(data_products_by_obs))
products_asn = Observations.filter_products(data_products_by_obs, extension="json")
print("products_asn length:", len(products_asn))
#valid_obsids = products_asn['obs_id'][np.char.find(np.unique(products_asn['obs_id']), 'jw02221-o001', ) == 0]
#match = [x for x in valid_obsids if filtername.lower() in x][0]
asn_mast_data = products_asn#[products_asn['obs_id'] == match]
print("asn_mast_data:", asn_mast_data)
manifest = Observations.download_products(asn_mast_data, download_dir=output_dir)
print("manifest:", manifest)

2023-09-06T13:07:43.456939: Obs table length: 15
2023-09-06T13:07:47.859042: data prodcts by obs length:  29708


2023-09-06 13:07:48,368 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221-o001_t001_nircam_f405n-f444w/jw02221-o001_20230725t121309_image3_00001_asn.json with expected size 15301.


2023-09-06T13:07:48.318048: products_asn length: 82
2023-09-06T13:07:48.318584: asn_mast_data:   obsID   obs_collection dataproduct_type                obs_id                ...  size parent_obsid    dataRights    calib_level
--------- -------------- ---------------- ------------------------------------ ... ----- ------------ ---------------- -----------
 90191617           JWST            image jw02221-o001_t001_nircam_f405n-f444w ... 15301     90191617           PUBLIC           3
127345652           JWST            image jw02221-o002_t001_nircam_f405n-f444w ... 10149    127345652 EXCLUSIVE_ACCESS           3
 90106375           JWST            image   jw02221001001_03101_00001_nrcalong ...  1812     90191617           PUBLIC           2
 90111555           JWST            image   jw02221001001_03101_00001_nrcblong ...  1812     90191617           PUBLIC           2
 90103980           JWST            image   jw02221001001_03101_00002_nrcalong ...  1812     90191617           PUBLIC 

2023-09-06 13:07:48,522 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221-o002_t001_nircam_f405n-f444w/jw02221-o002_20230725t121309_image3_00005_asn.json with expected size 10149.
2023-09-06 13:07:48,675 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221001001_03101_00001_nrcalong/jw02221-o001_20230725t121309_image2_00706_asn.json with expected size 1812.


INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221-o002_t001_nircam_f405n-f444w/jw02221-o002_20230725t121309_image3_00005_asn.json with expected size 10149. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221001001_03101_00001_nrcalong/jw02221-o001_20230725t121309_image2_00706_asn.json with expected size 1812. [astroquery.query]


...

INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221001001_03101_00024_nrcblong/jw02221-o001_20230725t121309_image2_00642_asn.json with expected size 1812. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00001_nrcalong/jw02221-o002_20230725t121309_image2_00475_asn.json with expected size 1812. [astroquery.query]


2023-09-06 13:07:56,235 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00001_nrcblong/jw02221-o002_20230725t121309_image2_00461_asn.json with expected size 1812.
2023-09-06 13:07:56,383 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00002_nrcalong/jw02221-o002_20230725t121309_image2_00500_asn.json with expected size 1812.


INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00001_nrcblong/jw02221-o002_20230725t121309_image2_00461_asn.json with expected size 1812. [astroquery.query]
INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00002_nrcalong/jw02221-o002_20230725t121309_image2_00500_asn.json with expected size 1812. [astroquery.query]


2023-09-06 13:07:56,536 - stpipe - INFO - Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00002_nrcblong/jw02221-o002_20230725t121309_image2_00451_asn.json with expected size 1812.


INFO: Found cached file /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221002001_02201_00002_nrcblong/jw02221-o002_20230725t121309_image2_00451_asn.json with expected size 1812. [astroquery.query]
2023-09-06T13:08:02.863904: manifest:                                                                         Local Path                                                                        ...
--------------------------------------------------------------------------------------------------------------------------------------------------------- ...
/orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221-o001_t001_nircam_f405n-f444w/jw02221-o001_20230725t121309_image3_00001_asn.json ...
/orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221-o002_t001_nircam_f405n-f444w/jw02221-o002_20230725t121309_image3_00005_asn.json ...
  /orange/adamginsburg/jwst/cloudc/F405N/pipeline/mastDownload/JWST/jw02221001001_03101_00001_nrcalong/jw02221

In [5]:
# MAST creates deep directory structures we don't want
for row in manifest:
    try:
        shutil.move(row['Local Path'], os.path.join(output_dir, os.path.basename(row['Local Path'])))
    except Exception as ex:
        print(f"Failed to move file with error {ex}")


In [6]:

products_fits = Observations.filter_products(data_products_by_obs, extension="fits")
print("products_fits length:", len(products_fits))
uncal_mask = np.array([uri.endswith('_uncal.fits') and f'jw02221{field}' in uri for uri in products_fits['dataURI']])
uncal_mask &= products_fits['productType'] == 'SCIENCE'
print("uncal length:", (uncal_mask.sum()))

already_downloaded = np.array([os.path.exists(os.path.basename(uri)) for uri in products_fits['dataURI']])
uncal_mask &= ~already_downloaded
print(f"uncal to download: {uncal_mask.sum()}; {already_downloaded.sum()} were already downloaded")

2023-09-06T13:08:03.368674: products_fits length: 29060
2023-09-06T13:08:03.804510: uncal length: 32
2023-09-06T13:08:06.749723: uncal to download: 0; 274 were already downloaded


In [7]:
field

'002'

In [8]:
products_fits[(products_fits['productType'] == 'SCIENCE') & np.array([uri.endswith('_uncal.fits') for uri in products_fits['dataURI']])]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str6,str4,str51,int64,str9,str16,int64
90106375,JWST,image,jw02221001001_03101_00001_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00001_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00001_nrcalong_uncal.fits,25225920,90191617,PUBLIC,1
90111555,JWST,image,jw02221001001_03101_00001_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00001_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00001_nrcblong_uncal.fits,25225920,90191617,PUBLIC,1
90103980,JWST,image,jw02221001001_03101_00002_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00002_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00002_nrcalong_uncal.fits,25225920,90191617,PUBLIC,1
90112033,JWST,image,jw02221001001_03101_00002_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00002_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00002_nrcblong_uncal.fits,25225920,90191617,PUBLIC,1
90107860,JWST,image,jw02221001001_03101_00003_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00003_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00003_nrcalong_uncal.fits,25225920,90191617,PUBLIC,1
90112704,JWST,image,jw02221001001_03101_00003_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00003_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00003_nrcblong_uncal.fits,25225920,90191617,PUBLIC,1
90104009,JWST,image,jw02221001001_03101_00004_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00004_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00004_nrcalong_uncal.fits,25225920,90191617,PUBLIC,1
90112856,JWST,image,jw02221001001_03101_00004_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00004_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00004_nrcblong_uncal.fits,25225920,90191617,PUBLIC,1
90107966,JWST,image,jw02221001001_03101_00005_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_03101_00005_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_03101_00005_nrcalong_uncal.fits,25225920,90191617,PUBLIC,1


In [9]:
from align_to_catalogs import retrieve_vvv

In [2]:
retrieve_vvv(
    basepath = '/orange/adamginsburg/jwst/cloudc/',
    filtername = 'f405n',
    module = 'merged',
    imfile = None,
    catfile = None,
    fov_regname='regions/nircam_cloudc_fov.reg',
    fieldnumber='002',
)

(<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
     [(266.581129, -28.666482), (266.594517, -28.666434),
      (266.604305, -28.663877), ..., (266.555803, -28.513283),
      (266.560494, -28.514834), (266.559398, -28.51401 )]>,
 <Table length=9945>
  RAJ2000    DEJ2000           iauname          mClass  Zmag3   e_Zmag3   Zperrbits  Ymag3  e_Ymag3  ...  Hmag3  e_Hmag3  Hperrbits  Ksmag3 e_Ksmag3 Ksperrbits     RA        DEC            skycoord      
    deg        deg                                       mag      mag                 mag     mag    ...   mag     mag                mag     mag                  deg        deg            deg,deg       
  float64    float64            str24           int16  float32  float64     int32   float64 float64  ... float64 float64    int32   float64 float64    int32     float64    float64          SkyCoord      
 ---------- ---------- ------------------------ ------ ------- ---------- --------- ------- -------- ... ------- -------- --------- 

In [9]:
#member['expname']

NameError: name 'member' is not defined

In [10]:
import shutil
from align_to_catalogs import realign_to_vvv, realign_to_catalog, merge_a_plus_b, retrieve_vvv


In [7]:
realigned_vvv_filename = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_realigned-to-vvv.fits'
#f'{basepath}/{filtername.upper()}/pipeline/jw02221-o{field}_t001_nircam_clear-{filtername.lower()}-{module}_realigned-to-vvv.fits'
#shutil.copy(f'{basepath}/{filtername.upper()}/pipeline/jw02221-o{field}_t001_nircam_clear-{filtername.lower()}-{module}_i2d.fits',
#            realigned_vvv_filename)
shutil.copy('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_i2d.fits', realigned_vvv_filename)
filtername='f405n'
realigned = realign_to_vvv(filtername=filtername, fov_regname='regions/nircam_cloudc_fov.reg', basepath='/orange/adamginsburg/jwst/cloudc/', module='merged', fieldnumber='002',
                           imfile=realigned_vvv_filename, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9395 of 19507 sources meeting criteria mag<15 [align_to_catalogs]
2023-07-20T14:26:15.739478: Before realignment, offset is -0.005937815103607136 arcsec, -0.01007643522541457 arcsec.  Found 319 matches.
2023-07-20T14:26:15.746149: CRVAL before 266.5870709048492 -28.59030752349884
2023-07-20T14:26:15.751861: CRVAL after 266.58707255424 -28.590304724489
2023-07-20T14:26:16.012553: After realignment, offset is -0.015492051852561417 arcsec, -0.005075601654169759 arcsec with 320 mathces
2023-07-20T14:26:16.054517: After re-realignment, offset is -0.015492051852561417 arcsec, -0.005075601654169759 arcsec using 320 matches


In [3]:
from align_to_catalogs import realign_to_vvv, merge_a_plus_b, retrieve_vvv


In [87]:
fov_regname = {'brick': 'regions/nircam_brick_fov.reg',
              'cloudc': 'regions/nircam_cloudc_fov.reg',
              }
regionname = 'cloudc'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
filtername = 'F405N'
field = '002' 
module = 'nrca'
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'

In [88]:
fwhm_tbl = Table.read(f'{basepath}/reduction/fwhm_table.ecsv')
row = fwhm_tbl[fwhm_tbl['Filter'] == filtername]
fwhm = fwhm_arcsec = float(row['PSF FWHM (arcsec)'][0])
fwhm_pix = float(row['PSF FWHM (pixel)'][0])

In [89]:
print(f"Searching for {os.path.join(output_dir, f'jw02221-o{field}*_image3_*0[0-9][0-9]_asn.json')}")
asn_file_search = glob(os.path.join(output_dir, f'jw02221-o{field}*_image3_*0[0-9][0-9]_asn.json'))
if len(asn_file_search) == 1:
    asn_file = asn_file_search[0]
elif len(asn_file_search) > 1:
    asn_file = sorted(asn_file_search)[-1]
    print(f"Found multiple asn files: {asn_file_search}.  Using the more recent one, {asn_file}.")
else:
    raise ValueError(f"Mismatch: Did not find any asn files for module {module} for field {field} in {output_dir}")

2023-09-06T15:42:36.267652: Searching for /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002*_image3_*0[0-9][0-9]_asn.json
2023-09-06T15:42:36.271284: Found multiple asn files: ['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_asn.json', '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230410t190607_image3_00004_asn.json', '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_asn.json'].  Using the more recent one, /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_asn.json.


In [90]:
asn_file

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_asn.json'

In [91]:
asn_file_search

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_asn.json',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230410t190607_image3_00004_asn.json',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_asn.json']

In [92]:
mapping = crds.rmap.load_mapping(f'/orange/adamginsburg/jwst/{regionname}/crds/mappings/jwst/jwst_nircam_pars-tweakregstep_0003.rmap')
print(f"Mapping: {mapping.todict()['selections']}")
print(f"Filtername: {filtername}")
filter_match = [x for x in mapping.todict()['selections'] if filtername in x]
print(f"Filter_match: {filter_match} n={len(filter_match)}")
tweakreg_asdf_filename = filter_match[0][4]
tweakreg_asdf = asdf.open(f'https://jwst-crds.stsci.edu/unchecked_get/references/jwst/{tweakreg_asdf_filename}')
tweakreg_parameters = tweakreg_asdf.tree['parameters']
print(f'Filter {filtername}: {tweakreg_parameters}')

2023-09-06T15:42:37.255701: Mapping: [('NRC_IMAGE', 'F070W', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0030.asdf'), ('NRC_IMAGE', 'F090W', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0046.asdf'), ('NRC_IMAGE', 'F115W', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0039.asdf'), ('NRC_IMAGE', 'F140M', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0034.asdf'), ('NRC_IMAGE', 'F150W', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0041.asdf'), ('NRC_IMAGE', 'F150W2', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0050.asdf'), ('NRC_IMAGE', 'F150W2', 'F162M', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0055.asdf'), ('NRC_IMAGE', 'F150W2', 'F164N', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0033.asdf'), ('NRC_IMAGE', 'F182M', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_pars-tweakregstep_0047.asdf'), ('NRC_IMAGE', 'F187N', 'CLEAR', '2000-01-01 00:00:00', 'jwst_nircam_

In [93]:
#os.path.join(output_dir, member['expname'])

In [94]:
import astropy.units as u

In [95]:
row

Filter,Wavelength(µm),PSF FWHM (arcsec),PSF FWHM (pixel)
str6,float64,float64,float64
F405N,4.052,0.136,2.165


In [96]:
if module in ('nrca', 'nrcb'):
    print(f"Filter {filtername} module {module}")

    with open(asn_file) as f_obj:
        asn_data = json.load(f_obj)
    asn_data['products'][0]['name'] = f'jw02221-o{field}_t001_nircam_clear-{filtername.lower()}-{module}'
    asn_data['products'][0]['members'] = [row for row in asn_data['products'][0]['members']
                                            if f'{module}' in row['expname']]

    for member in asn_data['products'][0]['members']:
        print(member['expname'])
        member['expname'] = os.path.join(output_dir, member['expname'])
        hdr = fits.getheader(member['expname'])
        if filtername in (hdr['PUPIL'], hdr['FILTER']):
            outname = destreak(member['expname'],
                               use_background_map=True,
                               median_filter_size=2048)  # median_filter_size=medfilt_size[filtername])
            member['expname'] = outname

        print(member['expname'])
        if field == '002' and (filtername.lower() == 'f405n' or filtername.lower() == 'f410m' or filtername.lower() == 'f466n'):
                align_image = member['expname'].replace("_destreak.fits", "_align.fits")#.split('.')[0]+'_align.fits'
                shutil.copy(member['expname'], align_image)
                offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
                row = offsets_tbl[member['expname'].split('/')[-1] == offsets_tbl['Filename_1']]
                align_fits = fits.open(align_image)
                pixel_scale = np.sqrt(fits.getheader(align_image, ext=1)['PIXAR_A2']*u.arcsec**2)
                try: 
                    print('Running manual align.')
                    print(align_fits['SCI',1].header['CRPIX1'], (float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX1'] + (float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX2'], (float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX2'] + (float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    align_fits['SCI',1].header['CRPIX1']+=(float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value
                    align_fits['SCI',1].header['CRPIX2']+=(float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value
                except: 
                    print('Something went wrong with manual align, running default values.')
                    visit = member['expname'].split('_')[0][-3:]
                    if visit == '001':
                        align_fits['SCI',1].header['CRPIX1']+=(8*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(-0.3*u.arcsec/pixel_scale).value
                    elif visit == '002':
                        align_fits['SCI',1].header['CRPIX1']+=(3.9*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(1*u.arcsec/pixel_scale).value
                    else:
                        align_fits['SCI',1].header['CRPIX1']+=(0*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(0*u.arcsec/pixel_scale).value
                align_fits.writeto(align_image, overwrite=True)
                member['expname'] = align_image

    asn_file_each = asn_file.replace("_asn.json", f"_{module}_asn.json")
    with open(asn_file_each, 'w') as fh:
        json.dump(asn_data, fh)
        
if module == 'merged':
        # try merging all frames & modules
        log.info("Working on merged reduction (both modules)")

        fov_regname = {'brick': 'regions/nircam_brick_fov.reg',
                       'cloudc': 'regions/nircam_cloudc_fov.reg',
                      }

        # Load asn_data for both modules
        with open(asn_file) as f_obj:
            asn_data = json.load(f_obj)

        for member in asn_data['products'][0]['members']:
            hdr = fits.getheader(member['expname'])
            if filtername in (hdr['PUPIL'], hdr['FILTER']):
                outname = destreak(member['expname'],
                                   use_background_map=True,
                                   median_filter_size=2048)  # median_filter_size=medfilt_size[filtername])
                member['expname'] = outname
            
            if field == '002' and (filtername.lower() == 'f405n' or filtername.lower() == 'f410m' or filtername.lower() == 'f466n'):
                align_image = member['expname'].replace("_destreak.fits", "_align.fits")#.split('.')[0]+'_align.fits'
                shutil.copy(member['expname'], align_image)
                offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
                row = offsets_tbl[member['expname'].split('/')[-1] == offsets_tbl['Filename_1']]
                align_fits = fits.open(align_image)
                pixel_scale = np.sqrt(fits.getheader(align_image, ext=1)['PIXAR_A2']*u.arcsec**2)
                try: 
                    print('Running manual align.')
                    print(align_fits['SCI',1].header['CRPIX1'], (float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX1'] + (float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX2'], (float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    print(align_fits['SCI',1].header['CRPIX2'] + (float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value)
                    align_fits['SCI',1].header['CRPIX1']+=(float(row['xshift (arcsec)'])*u.arcsec/pixel_scale).value
                    align_fits['SCI',1].header['CRPIX2']+=(float(row['yshift (arcsec)'])*u.arcsec/pixel_scale).value
                except: 
                    print('Something went wrong with manual align, running default values.')
                    visit = member['expname'].split('_')[0][-3:]
                    if visit == '001':
                        align_fits['SCI',1].header['CRPIX1']+=(8*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(-0.3*u.arcsec/pixel_scale).value
                    elif visit == '002':
                        align_fits['SCI',1].header['CRPIX1']+=(3.9*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(1*u.arcsec/pixel_scale).value
                    else:
                        align_fits['SCI',1].header['CRPIX1']+=(0*u.arcsec/pixel_scale).value
                        align_fits['SCI',1].header['CRPIX2']+=(0*u.arcsec/pixel_scale).value
                align_fits.writeto(align_image, overwrite=True)
                member['expname'] = align_image

        asn_data['products'][0]['name'] = f'jw02221-o{field}_t001_nircam_clear-{filtername.lower()}-merged'
        asn_file_merged = asn_file.replace("_asn.json", f"_merged_asn.json")
        with open(asn_file_merged, 'w') as fh:
            json.dump(asn_data, fh)

2023-09-06T15:42:39.152788: Filter F405N module nrca
2023-09-06T15:42:39.154592: jw02221002002_08201_00004_nrcalong_cal.fits
Destreaking /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00004_nrcalong_cal.fits


2023-09-06 15:42:39,859 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/miniconda3/envs/python39savannah/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1577: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,

2023-09-06 15:42:40,836 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:819: FITSFixedWarning: 'datfix' made the change 'Set DATE-BEG to '2023-04-25T05:05:59.722' from MJD-BEG.
Set DATE-AVG to '2023-04-25T05:06:26.564' from MJD-AVG.
Set DATE-END to '2023-04-25T05:06:53.406' from MJD-END'.
  warnings.warn(

2023-09-06 15:42:40,838 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:819: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to  -119.975053 from OBSGEO-[XYZ].
Set OBSGEO-B to   -11.515215 from OBSGEO-[XYZ].
Set OBSGEO-H to 1386464604.607 from OBSGEO-[XYZ]'.
  warnings.warn(

2023-09-06 15:42:40,848 - stpipe - WARNING - /blue/adamgi

2023-09-06T15:42:41.267623: /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00004_nrcalong_destreak.fits


2023-09-06 15:42:41,537 - stpipe - WARNING - /scratch/local/9079035/ipykernel_3543302/174679494.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='jw02221002001_02201_00004_nrcalong_align.fits'>
  align_fits = fits.open(align_image)



2023-09-06T15:42:41.568305: Running manual align.
2023-09-06T15:42:41.569847: 1024.5 63.10377574671999
2023-09-06T15:42:41.569985: 1087.60377574672
2023-09-06T15:42:41.570141: 1024.5 15.975639429549364
2023-09-06T15:42:41.570258: 1040.4756394295493
2023-09-06T15:42:41.728189: jw02221002002_08201_00003_nrcalong_cal.fits
Destreaking /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00003_nrcalong_cal.fits


...

2023-09-06T15:43:16.460528: Running manual align.
2023-09-06T15:43:16.462282: 1024.5 126.20755149343998
2023-09-06T15:43:16.462422: 1150.70755149344
2023-09-06T15:43:16.462590: 1024.5 -4.792691828864809
2023-09-06T15:43:16.462709: 1019.7073081711352
2023-09-06T15:43:16.728288: jw02221002001_02201_00004_nrcalong_cal.fits
Destreaking /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_cal.fits


2023-09-06 15:43:18,442 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:819: FITSFixedWarning: 'datfix' made the change 'Set DATE-BEG to '2023-04-15T15:16:01.022' from MJD-BEG.
Set DATE-AVG to '2023-04-15T15:16:27.864' from MJD-AVG.
Set DATE-END to '2023-04-15T15:16:54.706' from MJD-END'.
  warnings.warn(

2023-09-06 15:43:18,444 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/repos/astropy/astropy/wcs/wcs.py:819: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to  -134.370854 from OBSGEO-[XYZ].
Set OBSGEO-B to    -3.182913 from OBSGEO-[XYZ].
Set OBSGEO-H to 1298851117.169 from OBSGEO-[XYZ]'.
  warnings.warn(

2023-09-06 15:43:18,924 - stpipe - WARNING - /scratch/local/9079035/ipykernel_3543302/174679494.py:15: ResourceWarning: unclosed file <_io.BufferedReader name='/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_cal.fits'>
  outname = destreak(member['expname'],



2023-09-06T15:43:18.930350: /orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_destreak.fits


2023-09-06 15:43:19,192 - stpipe - WARNING - /scratch/local/9079035/ipykernel_3543302/174679494.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00002_nrcalong_align.fits'>
  align_fits = fits.open(align_image)



2023-09-06T15:43:19.223872: Running manual align.
2023-09-06T15:43:19.225468: 1024.5 126.20755149343998
2023-09-06T15:43:19.225606: 1150.70755149344
2023-09-06T15:43:19.225763: 1024.5 -4.792691828864809
2023-09-06T15:43:19.225882: 1019.7073081711352


In [22]:
asn_file#['products'][0]['members']

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_asn.json'

In [18]:
asn_file_each

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_nrca_asn.json'

In [97]:

if filtername.lower() == 'f405n':
# for the VVV cat, use the merged version: no need for independent versions
    abs_refcat = vvvdr2fn = (f'{basepath}/{filtername.upper()}/pipeline/jw02221-o{field}_t001_nircam_clear-{filtername}-merged_vvvcat.ecsv') # file needed by crowdsource_catalogs_long
    print(f"Loaded VVV catalog {vvvdr2fn}")
    retrieve_vvv(basepath=basepath, filtername=filtername, fov_regname=fov_regname[regionname], module='merged', fieldnumber=field)
    tweakreg_parameters['abs_refcat'] = vvvdr2fn
    tweakreg_parameters['abs_searchrad'] = 1
    reftbl = Table.read(abs_refcat)
    assert 'skycoord' in reftbl.colnames
else:
    abs_refcat = f'{basepath}/catalogs/crowdsource_based_nircam-f405n_reference_astrometric_catalog.ecsv'
    reftbl = Table.read(abs_refcat)
    # For non-F410M, try aligning to F410M instead of VVV?
    reftblversion = reftbl.meta['VERSION']

    # truncate to top 10,000 sources
    reftbl[:10000].write(f'{basepath}/catalogs/crowdsource_based_nircam-f405n_reference_astrometric_catalog_truncated10000.ecsv', overwrite=True)
    abs_refcat = f'{basepath}/catalogs/crowdsource_based_nircam-f405n_reference_astrometric_catalog_truncated10000.ecsv'

    tweakreg_parameters['abs_searchrad'] = 0.4
    # try forcing searchrad to be tighter to avoid bad crossmatches
    # (the raw data are very well-aligned to begin with, though CARTA
    # can't display them b/c they are using SIP)
    tweakreg_parameters['searchrad'] = 0.05
    print(f"Reference catalog is {abs_refcat} with version {reftblversion}")


2023-09-06T15:43:19.444221: Loaded VVV catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.ecsv


In [56]:

tweakreg_parameters.update({
                           'fitgeometry': 'general',
                           # brightest = 5000 was causing problems- maybe the cross-alignment was getting caught on PSF artifacts?
                           'brightest': 500,
                           'snr_threshold': 30, # was 5, but that produced too many stars
                           'abs_refcat': abs_refcat,
                           'save_catalogs': True,
                           'catalog_format': 'ecsv',
                           'kernel_fwhm': fwhm_pix,
                           'nclip': 5,
                           # based on DebugReproduceTweakregStep
                           'sharplo': 0.3,
                           'sharphi': 0.9,
                           'roundlo': -0.25,
                           'roundhi': 0.25,
                           #'searchrad': 4.0,
                           #'xoffset': -6.0,
                           'separation': 0.5, # minimum separation; default is 1
                           'skip': True
                           # 'clip_accum': True, # https://github.com/spacetelescope/tweakwcs/pull/169/files
                            
                           })

log.info(f"Running tweakreg ({module})")
calwebb_image3.Image3Pipeline.call(
    asn_file_each,
    steps={'tweakreg': tweakreg_parameters,},
    output_dir=output_dir,
    save_results=True)
print(f"DONE running {asn_file_each}")

2023-09-01 18:14:50,648 - stpipe - INFO - Running tweakreg (nrcb)


INFO: Running tweakreg (nrcb) [unknown]


2023-09-01 18:14:54,295 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /orange/adamginsburg/jwst/cloudc//crds/references/jwst/nircam/jwst_nircam_pars-tweakregstep_0035.asdf
2023-09-01 18:14:54,334 - CRDS - ERROR -  Error determining best reference for 'pars-sourcecatalogstep'  =   No match found.
2023-09-01 18:14:54,351 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2023-09-01 18:14:54,354 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2023-09-01 18:14:54,357 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2023-09-01 18:14:54,362 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2023-09-01 18:14:54,365 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-09-01 18:14:54,368 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2023-09-01 18:14:54,371 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instan

KeyboardInterrupt: 

In [13]:
output_dir

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/'

In [98]:
tweakreg = TweakRegStep()
tweakreg.fitgeometry = 'general'
tweakreg.brightest = 500
tweakreg.snr_threshold = 30
tweakreg.abs_refcat = abs_refcat
tweakreg.save_catalogs = True
tweakreg.catalog_format = 'ecsv'
tweakreg.kernel_fwhm = fwhm_pix
tweakreg.nclip = 5
tweakreg.sharplo = 0.3
tweakreg.sharphi = 0.9
tweakreg.roundlo = -0.25
tweakreg.roundhi = 0.25
tweakreg.separation = 0.5
tweakreg.output_dir = output_dir
tweakreg.save_results = True
tweakreg.skip = True
#tweakreg.searchrad = 4.0
#tweakreg.xoffset = -6.0
tweak = tweakreg.run(asn_file_each)

2023-09-06 15:43:20,219 - stpipe.TweakRegStep - INFO - TweakRegStep instance created.
2023-09-06 15:43:20,426 - stpipe.TweakRegStep - INFO - Step TweakRegStep running with args ('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_nrca_asn.json',).
2023-09-06 15:43:20,430 - stpipe.TweakRegStep - INFO - Step TweakRegStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/', 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': True, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_catalogs': True, 'use_custom_catalogs': False, 'catalog_format': 'ecsv', 'catfile': '', 'kernel_fwhm': 2.165, 'snr_threshold': 30, 'sharplo': 0.3, 'sharphi': 0.9, 'roundlo': -0.25, 'roundhi': 0.25, 'brightest': 500, 'peakmax': None, 'bkg_boxsize': 400, 'enforce_user_order': False, 'expand_refcat': False, 'minobj': 15, 'se

In [55]:
tweak_file = os.path.join(output_dir, tweak[0].meta.filename)
tweak_file

AttributeError: 'str' object has no attribute 'meta'

In [59]:
def compare(files):
    x, y = (1024, 1024)
    for f in files:
        cal_file = f
        tweak_file = f[:-8]+'destreak_tweakregstep.fits'
        
        cal_data = datamodels.open(cal_file)
        tweak_data = datamodels.open(tweak_file)
        
        cal_detector_to_world = cal_data.meta.wcs.get_transform('detector', 'world')
        run_detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')
        
        print('Cal:', cal_detector_to_world(x, y))
        print(run_detector_to_world(x, y))
        diffx = cal_detector_to_world(x, y)[0] - run_detector_to_world(x, y)[0]
        diffy = cal_detector_to_world(x, y)[1] - run_detector_to_world(x, y)[1]
        print('Difference:', diffx, diffy)
        
        print()
        

point = '02201'
files = glob(os.path.join(output_dir, f'jw02221{field}*{point}*{module}long_cal.fits'))
compare(files)

2023-09-01T18:15:43.481795: Cal: (266.57889003017374, -28.641290897929444)
2023-09-01T18:15:43.485653: (266.5790276601067, -28.63981286055749)
2023-09-01T18:15:43.498388: Difference: -0.00013762993296495551 -0.001478037371953178
2023-09-01T18:15:43.498443:
2023-09-01T18:15:45.267377: Cal: (266.57782016520355, -28.64124472704071)
2023-09-01T18:15:45.270898: (266.57795771862146, -28.639766169654745)
2023-09-01T18:15:45.282605: Difference: -0.0001375534179146598 -0.0014785573859654733
2023-09-01T18:15:45.282649:
2023-09-01T18:15:47.064634: Cal: (266.58641178175856, -28.60560461822254)
2023-09-01T18:15:47.068338: (266.5868443193349, -28.60454317273278)
2023-09-01T18:15:47.079630: Difference: -0.00043253757633010537 -0.0010614454897606151
2023-09-01T18:15:47.079672:


KeyboardInterrupt: 

In [64]:
cal_data = datamodels.open('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00002_nrcalong_cal.fits')
#tweak_data = datamodels.open(tweak_file)
tweak_data = datamodels.open('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00002_nrcalong_destreak.fits')


In [66]:
cal_detector_to_world = cal_data.meta.wcs.get_transform('detector', 'world')
run_detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')

In [68]:
print(cal_detector_to_world(x, y))
print(run_detector_to_world(x, y))

2023-07-27T14:16:44.932300: (266.58459696150726, -28.59355794836456)
2023-07-27T14:16:44.951258: (266.58459696150726, -28.59355794836456)


In [58]:
run_detector_to_world

<CompoundModel(offset_0=-0.093, offset_1=0.033, offset_2=1., offset_3=1., offset_4=-1024.5, offset_5=-1024.5, c0_0_7=0., c1_0_7=0.06275847, c2_0_7=0.00000012, c3_0_7=0., c4_0_7=-0., c5_0_7=0., c0_1_7=-0., c0_2_7=-0.0000001, c0_3_7=-0., c0_4_7=0., c0_5_7=0., c1_1_7=-0.00000073, c1_2_7=0., c1_3_7=-0., c1_4_7=0., c2_1_7=-0., c2_2_7=0., c2_3_7=0., c3_1_7=-0., c3_2_7=0., c4_1_7=0., c0_0_8=0., c1_0_8=-0.00009902, c2_0_8=0.00000031, c3_0_8=0., c4_0_8=0., c5_0_8=-0., c0_1_8=0.06306468, c0_2_8=-0.00000042, c0_3_8=0., c0_4_8=-0., c0_5_8=0., c1_1_8=0.00000023, c1_2_8=-0., c1_3_8=0., c1_4_8=-0., c2_1_8=0., c2_2_8=-0., c2_3_8=0., c3_1_8=0., c3_2_8=-0., c4_1_8=0., c0_0_10=0., c1_0_10=-0.99999954, c0_1_10=-0.00095806, c0_0_11=0., c1_0_11=-0.00095806, c0_1_11=0.99999954, offset_12=85.9392976, offset_13=-493.5106528, factor_14=1.00008927, factor_15=1.00008927, offset_16=-0.0076718, offset_17=0.04405571, factor_18=0.00027778, factor_19=0.00027778, angles_21=[   0.02387203,    0.13708629,  275.45960573, 

In [60]:
tweak_data.meta.wcs

NameError: name 'tweak_data' is not defined

In [69]:
glob(os.path.join(output_dir, f'jw02221-o{field}*_image3_*0[0-9][0-9]_asn.json'))

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230410t190607_image3_00004_asn.json',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_asn.json']

In [76]:
print('test')

2023-07-27T14:39:46.562408: test


In [99]:
skymatch = SkyMatchStep()
skymatch.skymethod = 'global+match'
skymatch.save_results = True
skymatch.output_dir = output_dir
sky = skymatch.run(tweak)

2023-09-06 15:43:20,445 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.
2023-09-06 15:43:20,634 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230725t121309_image3_00005_nrca_asn.json',).
2023-09-06 15:43:20,638 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'global+match', 'match_down': True, 'subtract': False, 'stepsize': None, 'skystat': 'mode', 'dqbits': '~DO_NOT_USE+NON_SCIENCE', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}
2023-09-06 15:43:20,660 - stpipe.SkyMatchStep - WARNING - /blue/adamginsburg/adamginsburg/minico

! WARNING: Clipped data falls within 1 histogram bin
! WARNING: Clipped data falls within 1 histogram bin


2023-09-06 15:45:18,181 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_08201_00004_nrcalong_align.fits. Sky background: 0
2023-09-06 15:45:18,183 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_08201_00003_nrcalong_align.fits. Sky background: 0.00905509
2023-09-06 15:45:18,184 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_08201_00001_nrcalong_align.fits. Sky background: 0.124691
2023-09-06 15:45:18,186 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002001_08201_00003_nrcalong_align.fits. Sky background: 0.937662
2023-09-06 15:45:18,187 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_08201_00002_nrcalong_align.fits. Sky background: 0.0156086
2023-09-06 15:45:18,188 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_02201_00001_nrcalong_align.fits. Sky background: 0.540792
2023-09-06 15:45:18,190 - stpipe.SkyMatchStep - INFO -    *  Image ID=jw02221002002_02201_00003_nrcalong_align.fits. Sky background: 0.55995
2023-09-06 

In [101]:
for i in range(16):
    print(sky[0].meta.filename)

2023-07-27T18:29:08.056126: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056256: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056277: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056294: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056309: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056325: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056340: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056355: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056371: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056386: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056400: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056415: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056430: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056445: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27T18:29:08.056461: step_SkyMatchStep_0_skymatchstep.fits
2023-07-27

In [62]:
sky_file = sky[0].meta.filename
sky_header = fits.getheader(sky_file)
print('Computed background level: {}'.format(sky_header['BKGLEVEL']))
print('Background subtracted: {}'.format(sky_header['BKGSUB']))

2023-09-01T18:23:07.804229: Computed background level: 3.1323486857077834
2023-09-01T18:23:07.804544: Background subtracted: False


In [63]:
sky_data = datamodels.open(sky_file)

In [65]:
np.nanmin(sky_data.data - cal_data.data), np.nanmax(sky_data.data - cal_data.data) # difference is probably due to destreaking 

(-28818.928, 21596.38)

In [100]:
outlier_detection = OutlierDetectionStep()
outlier_detection.save_results = True
outlier_detection.save_intermediate_results = True
outlier = outlier_detection.run(sky)

2023-09-06 15:45:28,472 - stpipe.OutlierDetectionStep - INFO - OutlierDetectionStep instance created.
2023-09-06 15:45:28,731 - stpipe.OutlierDetectionStep - INFO - Step OutlierDetectionStep running with args (<ModelContainer>,).
2023-09-06 15:45:28,734 - stpipe.OutlierDetectionStep - INFO - Step OutlierDetectionStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': False, 'input_dir': '', 'weight_type': 'ivm', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'nlow': 0, 'nhigh': 0, 'maskpt': 0.7, 'grow': 1, 'snr': '5.0 4.0', 'scale': '1.2 0.7', 'backg': 0.0, 'kernel_size': '7 7', 'threshold_percent': 99.8, 'ifu_second_check': False, 'save_intermediate_results': True, 'resample_data': True, 'good_bits': '~DO_NOT_USE', 'scale_detection': False, 'allowed_memory': None, 'in_memory': False}
20

In [86]:
glob('*outlier*i2d.fits')

['jw02221002001_02201_00004_nrcblong_outlier_i2d.fits',
 'jw02221002001_08201_00003_nrcalong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002001_02201_00003_nrcblong_outlier_i2d.fits',
 'jw02221002002_08201_00001_nrcblong_outlier_i2d.fits',
 'step_SkyMatchStep_5_outlier_i2d.fits',
 'jw02221002001_02201_00003_nrcalong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002001_02201_00001_nrcalong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002002_08201_00003_nrcblong_outlier_i2d.fits',
 'step_SkyMatchStep_10_outlier_i2d.fits',
 'jw02221002001_02201_00004_nrcblong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002001_02201_00001_nrcalong_outlier_i2d.fits',
 'jw02221002001_02201_00003_nrcblong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002002_08201_00003_nrcalong_destreak_realigned_to_outlier_i2d.fits',
 'step_SkyMatchStep_3_outlier_i2d.fits',
 'jw02221002001_02201_00002_nrcblong_destreak_realigned_to_outlier_i2d.fits',
 'jw02221002001_08201_00002_nrcblong_destreak_real

In [59]:
print('DONE')

2023-09-06T13:32:42.092035: DONE


In [115]:
out_lis = np.array(['15', '10', '9', '8', '7', '6', '5', '4'])

In [123]:
glob('step_SkyMatchStep_*_outlier_i2d.fits')

['step_SkyMatchStep_5_outlier_i2d.fits',
 'step_SkyMatchStep_10_outlier_i2d.fits',
 'step_SkyMatchStep_3_outlier_i2d.fits',
 'step_SkyMatchStep_14_outlier_i2d.fits',
 'step_SkyMatchStep_11_outlier_i2d.fits',
 'step_SkyMatchStep_12_outlier_i2d.fits',
 'step_SkyMatchStep_1_outlier_i2d.fits',
 'step_SkyMatchStep_8_outlier_i2d.fits',
 'step_SkyMatchStep_15_outlier_i2d.fits',
 'step_SkyMatchStep_6_outlier_i2d.fits',
 'step_SkyMatchStep_2_outlier_i2d.fits',
 'step_SkyMatchStep_4_outlier_i2d.fits',
 'step_SkyMatchStep_0_outlier_i2d.fits',
 'step_SkyMatchStep_9_outlier_i2d.fits',
 'step_SkyMatchStep_7_outlier_i2d.fits',
 'step_SkyMatchStep_13_outlier_i2d.fits']